In [1]:
from funcs import funcs
from Crosswalk.Transformer import Transformer
from Crosswalk.DataCache import DataCache
from Crosswalk.NDAWriter import NDAWriter
from Crosswalk.Manager import Manager

from Crosswalk.Loader import BoxLoader, SsagaLoader, QintLoader, RedcapLoader, ParentLoader

In [2]:
M = Manager(
        data =  DataCache(
            ParentLoader(),
            RedcapLoader('child'),
            RedcapLoader('teen'),
            BoxLoader('PennCNP', 592325063896),
            QintLoader()
        ),
        writer = NDAWriter(completed_dir="./prepped/hcd/"),
        transformer = Transformer(funcs = funcs, map_dir='./maps/hcd/')
)

In [3]:
M.preload_data()

child Some columns were unavailable:  {'bld_rucdr_des', 'bld_core_des_'}
Timing:  child 1.5264739990234375
Timing:  parent 2.4297420978546143
Timing:  PennCNP 0.025960445404052734
teen Some columns were unavailable:  {'bld_rucdr_des_', 'bld_core_des_'}
Timing:  teen 1.0736656188964844
Timing:  qint 0.14849281311035156


In [5]:
M.run('leap01')
# data needs to be corrected in redcap.
# get subject/site and open trello ticket

For struct "leap01": 
lan3err: Invalid values  {130.0, 135.0, 200.0, 105.0, 175.0, 240.0, 181.0, 150.0, 120.0, 250.0}
Dropping 14 values.
lan4err: Invalid values  {200.0, 105.0, 300.0, 110.0, 175.0, 210.0, 180.0, 150.0, 120.0, 155.0}
Dropping 13 values.
No errors!


In [6]:
M.run('mctq01')
# trello

For struct "mctq01": 
frmnup: Invalid values  {2400, 1100}
Dropping 2 values.
No errors!


In [7]:
structs = [
    'asr01',
    'bisbas01',
    'bsc01',
    'cbcl01',
    'cbcl1_501',
    'cbq01',
    'drugscr01',
    'eatq01',
    'er4001',
    'fenvs01',
    'gbi01',
#     'leap01',
    'mab01',
#     'mctq01',
    'medh01',
    'mendt01',
    'mmse01',
    'neo_ffi_form_s_adult_200301',
    'pds01',
    'phenx_su01',
    'psqi01',
    'saiq01',
    'scan_debrief01',
    'screentime01',
    'sdq01',
    'sleepdis01',
    'socdem01',
    'srs02',
    'upps01',
    'vision_tests01',
    'vitals01',
    'wais_iv_part101',
    'wisc_v01',
    'wppsiiv01',
    'ysr01'
]

for s in structs:
    M.run(s)

For struct "asr01": 
No errors!
For struct "bisbas01": 
No errors!
For struct "bsc01": 
No errors!
For struct "cbcl01": 
No errors!
For struct "cbcl1_501": 
No errors!
For struct "cbq01": 
No errors!
For struct "drugscr01": 
No errors!
For struct "eatq01": 
No errors!
For struct "er4001": 
No errors!
For struct "fenvs01": 
No errors!
For struct "gbi01": 
No errors!
For struct "mab01": 
No errors!
For struct "medh01": 
No errors!
For struct "mendt01": 
No errors!
For struct "mmse01": 
No errors!
For struct "neo_ffi_form_s_adult_200301": 
No errors!
For struct "pds01": 
No errors!
For struct "phenx_su01": 
No errors!
For struct "psqi01": 
No errors!
For struct "saiq01": 
No errors!
For struct "scan_debrief01": 
No errors!
For struct "screentime01": 
No errors!
For struct "sdq01": 
No errors!
For struct "sleepdis01": 
No errors!
For struct "socdem01": 
No errors!
For struct "srs02": 
No errors!
For struct "upps01": 
No errors!
For struct "vision_tests01": 
No errors!
For struct "vitals01"